In [1]:


# Importing Necessary modules
from transformers import pipeline
from transformers import AutoTokenizer,RobertaTokenizer,TextClassificationPipeline
from optimum.onnxruntime import ORTModelForSequenceClassification
from fastapi import FastAPI
from scipy.special import softmax
from pydantic import BaseModel
from typing import List
import onnxruntime as ort
import numpy as np
from time import perf_counter
import uvicorn
import urllib
import csv

def preprocess(tweets):
    ptweets = []
    for tweet in tweets :
        new_text = []
        for t in tweet.split(" "):
            t = '@user' if t.startswith('@') and len(t) > 1 else t
            t = 'http' if t.startswith('http') else t
            new_text.append(t)
        new_text = " ".join(new_text)
        ptweets.append(new_text)
    return ptweets

model_path = 'EmoTwitter/app/twitter-models/base/'
tokenizer = AutoTokenizer.from_pretrained(model_path)
#The models are optimized for CPU inference, therefore no support for GPU execution is provided as of now
ort_session = ort.InferenceSession(model_path+'/model.onnx',providers=['DnnlExecutionProvider'])

text = ["I am very happy"]#,"Very very disaponted"]
text = preprocess(text)

inputs = tokenizer(text, add_special_tokens=True,return_tensors="np")

#convint = map()
inputs = {k: v.astype(np.int64) for k,v in inputs.items()} # handling in windows data type size
print('input : ',inputs)


outputs_name = ort_session.get_outputs()[0].name
outputs = ort_session.run(output_names=[outputs_name], input_feed=inputs)
scores = outputs[0][0]

labels = ['anger', 'joy', 'optimism', 'sadness']
ranking = np.argsort(scores)
ranking = ranking[::-1]
scores = softmax(scores)
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


c:\Users\amd\anaconda3\envs\mktmp\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'DnnlExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


input :  {'input_ids': array([[   0,  100,  524,  182, 1372,    2]], dtype=int64), 'attention_mask': array([[1, 1, 1, 1, 1, 1]], dtype=int64)}
1) joy 0.9179
2) optimism 0.0499
3) sadness 0.0225
4) anger 0.0097


In [6]:
inputs['input_ids'][0]

array([   0,  100,  524,  182, 1372,    2], dtype=int64)

In [11]:
text = ["I am very happy","Very very disaponted"]
text = preprocess(text)
labels = ['anger', 'joy', 'optimism', 'sadness']


model_path = 'EmoTwitter/app/twitter-models/base/'
tokenizer = AutoTokenizer.from_pretrained(model_path)
#The models are optimized for CPU inference, therefore no support for GPU execution is provided as of now
ort_session = ort.InferenceSession(model_path+'/model.onnx',providers=['DnnlExecutionProvider'])

for t in text :
    inputs = tokenizer(t, add_special_tokens=True,return_tensors="np")
    inputs = {k: [vi.astype(np.int64) for vi in v] for k,v in inputs.items()} # handling in windows data type size
    outputs_name = ort_session.get_outputs()[0].name
    outputs = ort_session.run(output_names=[outputs_name], input_feed=inputs)
    print(outputs)

    scores = outputs[0][0]

    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    scores = softmax(scores)
    opack = {}
    preds = {}
    opack['label'] = (labels[0],scores[0])
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        opack['logits'][l] = np.round(float(s), 4)
        print(f"{i+1}) {l} {np.round(float(s), 4)}")
    print("opakc ",opack)
    
#convint = map()
#print(inputs)
#inputs = {k: [vi.astype(np.int64) for vi in v] for k,v in inputs.items()} # handling in windows data type size
#print('input : ',inputs)

#outputs_name = ort_session.get_outputs()[0].name
#outputs = ort_session.run(output_names=[outputs_name], input_feed={'input_ids': [[   0,  100,  524,  182, 1372,    2]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]})
#outputs = ort_session.run(output_names=[outputs_name], input_feed=inputs)



c:\Users\amd\anaconda3\envs\mktmp\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'DnnlExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


[array([[-1.7885597 ,  2.7592015 , -0.15227139, -0.95142317]],
      dtype=float32)]
1) joy 0.9179
2) optimism 0.0499
3) sadness 0.0225
4) anger 0.0097
[array([[-0.5423151, -1.2919993, -1.5097198,  3.461708 ]], dtype=float32)]
1) sadness 0.9673
2) anger 0.0176
3) joy 0.0083
4) optimism 0.0067
